In [10]:
import sys
sys.path.append("../")

In [11]:
import numpy as np 		  	   		 	   			     			  	 
import pandas as pd
import datetime as dt
			     			  	 
from util import get_data

In [12]:
symbol = 'AAPL'
sd=dt.datetime(2010, 1, 1)
ed=dt.datetime(2011,12,31)
sv = 100000

In [15]:
stock_date_range = pd.date_range(sd, ed)
stock_date_range

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
               '2010-01-09', '2010-01-10',
               ...
               '2011-12-22', '2011-12-23', '2011-12-24', '2011-12-25',
               '2011-12-26', '2011-12-27', '2011-12-28', '2011-12-29',
               '2011-12-30', '2011-12-31'],
              dtype='datetime64[ns]', length=730, freq='D')

In [14]:
ticker_data = get_data(symbol, stock_date_range)

ValueError: columns overlap but no suffix specified: Index(['A'], dtype='object')